In [46]:
from facenet_pytorch import MTCNN
import cv2
import os
import pandas as pd
import numpy as np

In [50]:
detector = MTCNN()

def find_faces_in_videos(video_directory, save_directory, video_name, data):
    cap = cv2.VideoCapture(f'{video_directory}{video_name}')

    def get_frame(sec, current_count, name, label):
        cap.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
        hasFrames,image = cap.read()
        if hasFrames:
            faces = detector.detect(image)
            # Get typeof faces
            if type(faces[0]) == np.ndarray:
                for i in range(len(faces[0])):
                    try:
                        accuracy = faces[1][i]
                        if(accuracy > 0.99):
                            x1 = int(faces[0][i][0])
                            y1 = int(faces[0][i][1])
                            x2 = int(faces[0][i][2])
                            y2 = int(faces[0][i][3])
                            current_count += 1
                            face_image = image[y1:y2, x1:x2]

                            # Resize image
                            face_image = cv2.resize(face_image, (128, 128))

                            # Save image 
                            cv2.imwrite(f"{save_directory}{label}/{name}{count}.jpg", face_image)
                    except:
                        print("Some exception happened")
        return hasFrames, current_count

    label = data.loc[video_name, 'label']

    if not os.path.exists(f'{save_directory}{label}'):
        os.makedirs(f'{save_directory}{label}')

    sec = 0
    frame_rate = 2 #//it will capture image in each second
    count = 0
    name = video_name.replace(".mp4", "")
    success, count = get_frame(sec, 0, name, label)
    while success:
        success, count = get_frame(sec, count, name, label)
        sec += frame_rate

In [51]:
# Read frames from video
subdirs = ["dfdc_train_part_0"]

# Ensure directory exists
for dir in subdirs:
    path = f"./data/{dir}/"
    # Get all files in directory
    files = os.listdir(path)
    # Only keep .mp4 files
    files = [f for f in files if f.endswith(".mp4")]
    data = pd.read_json(f"./data/{dir}/metadata.json").T
    for file in files:
        find_faces_in_videos(path, "./data/processed/", file, data)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception happened
Some exception h